In [ ]:

import time

def task(name, seconds):
    print(f"{name}: Started")
    time.sleep(seconds)
    print(f"{name}: Finished after {seconds} sec.")


def main():
    task("Task A", 2)
    task("Task B", 3)
    task("Task 3", 1)

main()



In [ ]:

import asyncio

async def task(name, seconds):
    print(f"{name}: Started")
    await asyncio.sleep(seconds)
    print(f"{name}: Finished after {seconds} sec")

async def main():
    await asyncio.gather(
        task("Task A", 6),
        task("Task B", 5),
        task("Task C", 3)
    )

asyncio.run(main())

## Why Asynchronous Programming?

In traditional (synchronous) Python code, tasks are executed **one after the other**. If one task takes time (like waiting for a file to load or a server to respond), the entire program is **blocked** until it completes.

**Asynchronous programming** allows multiple tasks to run concurrently by "pausing" and "resuming" based on availability of resources (typically I/O). This leads to efficient usage of time and resources, especially when dealing with I/O-bound operations.

## Core Concepts

### 1. Coroutines

A **coroutine** is a special function that can **pause** and **resume** its execution.

Defined using:

```python
async def my_function():
    ...
```

Coroutines don’t run until they are awaited or scheduled in an event loop.

Example:

```python
async def greet():
    print("Hello")
    await asyncio.sleep(1)
    print("World")
```


### 2. `await`

`await` is used **inside an `async def` function** to **pause** that function until the awaited task completes. This only works with **awaitable** objects (usually other coroutines or `asyncio` methods).

Example:

```python
await asyncio.sleep(2)  # Pauses this coroutine for 2 seconds without blocking others
```

When a coroutine `await`s something:

- It yields control back to the event loop.
    
- The event loop continues executing other ready tasks.
    
- After the awaited operation finishes, control returns to the paused coroutine.
    

## Key Insight: Python's Async Model

Python's `asyncio` is **single-threaded** and based on an **event loop**.

- It runs coroutines concurrently using **cooperative multitasking**.
    
- Each coroutine **voluntarily gives up control** by using `await`.
    

This is not parallelism (like with threads), but **concurrent scheduling** of tasks.

## When Should You Use `await`?

Use `await` when you're dealing with **I/O-bound** tasks:

- Network requests (APIs)
    
- File I/O (if using async-friendly libraries)
    
- Database queries
    
- Timers or delays

Avoid using `await` for **CPU-bound** tasks (e.g., heavy computations) — these should use threads or processes.

|Task|Use `await`?|Reason|
|---|---|---|
|API Call|Yes|Waiting for server response|
|Async File Read|Yes|Disk I/O|
|`asyncio.sleep()`|Yes|Timed pause (non-blocking)|
|Heavy Computation|No|Blocks event loop|


In [1]:
import asyncio

async def task_a():
    print("Task A started")
    await asyncio.sleep(3)
    print("Task A finished")

async def task_b():
    print("Task B started")
    await asyncio.sleep(2)
    print("Task B finished")

await asyncio.gather(task_a(), task_b())

Task A started
Task B started
Task B finished
Task A finished


[None, None]

## Common Asyncio Tools

- `asyncio.run()` — runs the event loop for you
    
- `asyncio.gather()` — run multiple coroutines concurrently
    
- `asyncio.sleep()` — async version of `time.sleep()` (non-blocking)

In [ ]:
import requests
import time

urls = [
    "https://jsonplaceholder.typicode.com/posts/1",
    "https://jsonplaceholder.typicode.com/posts/2",
    "https://jsonplaceholder.typicode.com/posts/3",
    "https://jsonplaceholder.typicode.com/comments?postId=1",
    "https://jsonplaceholder.typicode.com/albums/1"
]



def fetch_data_sequential(urls):
    results = []
    start_time = time.time()
    
    for url in urls:
        response = requests.get(url)
        if response.status_code == 200:
            results.append(response.json())
    
    elapsed = time.time() - start_time
    print(f"Sequential requests completed in {elapsed:.2f} seconds")
    return results


results = fetch_data_sequential(urls)
print(f"Received {len(results)} responses")

Sequential requests completed in 0.24 seconds
Received 5 responses


In [ ]:

# create a .py file for the below code.

import time
import asyncio
import aiohttp


urls = [
    "https://jsonplaceholder.typicode.com/posts/1",
    "https://jsonplaceholder.typicode.com/posts/2",
    "https://jsonplaceholder.typicode.com/posts/3",
    "https://jsonplaceholder.typicode.com/comments?postId=1",
    "https://jsonplaceholder.typicode.com/albums/1"
]

async def fetch_url(session, url):
    async with session.get(url) as response:
        if response.status == 200:
            return await response.json()
        return None

async def fetch_data_async(urls):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_url(session, url) for url in urls]
        return await asyncio.gather(*tasks)

async def main():
    start_time = time.time()
    results = await fetch_data_async(urls)
    
    elapsed = time.time() - start_time
    print(f"Async requests completed in {elapsed:.2f} seconds")
    print(f"Received {len([r for r in results if r is not None])} responses")

if __name__ == "__main__":
   
    print("\nRunning async version now:")
    asyncio.run(main())

